Improved Naive RAG:
- image augmented dataset
- simple preprocessing
- optimized recursive chunking (size 1500, overlap 150)
- hybrid retriever - dense vectors and TF-IDF with k = 4 (sparse, weight 0.2) + 2 (dense, weight 0.8)
- bge-m3
- question re-writing
- reranking/filtering
- RRF
- prompting
- Gemini LLM

# Import libraries

In [ ]:
import pandas as pd
import pprint

# Import model

In [ ]:
import os
import google.generativeai as genai
from IPython.display import Markdown

genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    temperature=0
)

# Load dataset - vectorstore

In [ ]:
from FlagEmbedding import BGEM3FlagModel
from langchain_community.vectorstores import FAISS

model_fp16 = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

class M3EmbeddingFP16:
    def embed_documents(self, texts):
        return model_fp16.encode(texts)['dense_vecs']
    
    def __call__(self, texts):
        return self.embed_documents(texts)
    
embd = M3EmbeddingFP16()

In [ ]:
# Contains the documents without any data preprocessing steps
vectorstore = FAISS.load_local("cleaned_recursive_augmented_faiss_index", embd, allow_dangerous_deserialization=True)
vectorstore, vectorstore.index.ntotal

In [ ]:
path = os.getcwd()
csv_input_path = os.path.dirname(path) + "/augmented_dataset_final_outputs.csv"
# Read the CSV file into a DataFrame
df = pd.read_csv(csv_input_path, encoding='utf-8')

# Display the first few rows of the DataFrame to check the contents
display(df)

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(df, page_content_column="Text")
docs_data = loader.load()

import importlib
import Data_preprocessing
importlib.reload(Data_preprocessing)

# Initialize the Preprocessing object
preprocessing = Data_preprocessing.Preprocessing()

for doc in docs_data:
    doc.page_content = preprocessing.clean_text_template(doc.page_content)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)

splits = text_splitter.split_documents(docs_data)
pprint.pprint(splits[0:6])
pprint.pprint(len(splits))

In [ ]:
from langchain.retrievers import TFIDFRetriever, EnsembleRetriever

dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

sparse_retriever = TFIDFRetriever.from_documents(splits)
sparse_retriever.k =  4

retriever = EnsembleRetriever(retrievers=[sparse_retriever, dense_retriever], weights=[0.2, 0.8])

# RAG Fusion

## Question rewriting

In [ ]:
import pandas as pd
import pprint
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import os
from langchain_openai import ChatOpenAI
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

template = """
Sei un modello linguistico AI che svolge il ruolo di assistente clienti per il software Panthera. 
Il tuo compito è generare cinque versioni diverse della domanda fornita dall'utente per recuperare documenti rilevanti da un database vettoriale. 
Il contesto riguarda manuali tecnici di software per la gestione aziendale. Non fornire la riscrittura della domanda se non è relativa al contesto.
Se nessuna delle domande è rilevante per il contesto del software fornisci come output solamente il tag: "OUT-OF SCOPE".

**Esempi:**
1. Domanda originale: "Come posso specificare il tipo costo da assegnare al nuovo ambiente di commessa?"
   Risposte:
   - Qual è il procedimento per definire il tipo di costo da applicare a un nuovo ambiente di commessa?
   - Come posso configurare il tipo di costo per un nuovo ambiente di commessa nel sistema?
   - Quali sono i passaggi per assegnare un tipo di costo a un ambiente di commessa appena creato?
   - Dove posso trovare l'opzione per impostare il tipo di costo di un nuovo ambiente di commessa?
   - Come posso scegliere il tipo di costo da associare a un ambiente di commessa nel mio sistema?

2. Domanda originale: "Come si fa la carbonara?"
   Risposta: OUT-OF SCOPE

**Istruzioni:**
1. Genera domande riscritte che mantengano il significato originale, esplorando diverse formulazioni e angolazioni. 
2. Ignora le domande che non sono pertinenti ai manuali di software gestionale o agli argomenti di informatica.
3. Fornisci le domande alternative in un elenco puntato separato da nuove righe.
4. L'output deve contenere solo le domande riscritte, senza spiegazioni o commenti.

Svolgi la task solo per le domande rilevanti al contesto del software. In questo caso, cerca di migliorare la formulazione originale 
esplorando diverse angolazioni che aiutino a comprendere meglio il problema o la richiesta, rendendo più chiare e leggibili le domande per un utente generico. 

Domanda originale: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
model_gpt = ChatOpenAI(temperature=0, model="gpt-4o") 

generate_queries = (
    prompt 
    | model_gpt
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

## Reranking/Filtering

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi

def compute_cosine_similarity(embedding, embeddings):
    """Computes cosine similarity between one vector and a set of vectors."""
    # Compute the dot product between the single vector and each vector in embeddings
    dot_products = np.dot(embeddings, embedding)
    
    # Compute the norms
    norm_embedding = np.linalg.norm(embedding)
    norms_embeddings = np.linalg.norm(embeddings, axis=1)
    
    # Compute cosine similarities
    cosine_similarities = dot_products / (norm_embedding * norms_embeddings)
    
    return cosine_similarities


def compute_bm25_score(original, rewrites):
    """Computes BM25 scores for original question against rewritten questions."""
    # Prepare documents for BM25
    documents = [original] + rewrites
    tokenized_documents = [doc.split(" ") for doc in documents]
    
    # Initialize BM25
    bm25 = BM25Okapi(tokenized_documents)
    
    # Get BM25 scores for the original question against all rewrites
    scores = bm25.get_scores(tokenized_documents[0])
    
    return scores[1:]  # Exclude the score for the original question

def rerank_questions(original_question, alpha=0.5, threshold=1):
    """Rerank the rewritten questions based on cosine similarity and BM25 scores."""
    
    # Load a pre-trained model
    model = SentenceTransformer("BAAI/bge-m3")
    
    # Generate embeddings
    original_embedding = model.encode(original_question)
    rewritten_questions = generate_queries.invoke({"question": original_question})
    rewritten_embeddings = model.encode(rewritten_questions)

    # Convert list of embeddings to a numpy array
    rewritten_embeddings = np.vstack(rewritten_embeddings)  # Stack into a single 2D array
    
    # Compute cosine similarities
    cosine_similarities = compute_cosine_similarity(original_embedding, rewritten_embeddings)
    
    # Compute BM25 scores
    bm25_scores = compute_bm25_score(original_question, rewritten_questions)

    # Combine scores
    weighted_scores = alpha * cosine_similarities + (1 - alpha) * bm25_scores
    
    # Create a ranking of rewritten questions
    ranked_indices = np.argsort(weighted_scores)[::-1]  # Sort in descending order
    
    # Filter questions based on the threshold
    filtered_questions = [(rewritten_questions[i], weighted_scores[i]) for i in ranked_indices if weighted_scores[i] >= threshold]
    # EXPERIMENT IF IT WORKS BETTER WITH OR WITHOUT THE ORIGINAL QUESTION
    questions = []
    # questions = [original_question]
    for question, score in filtered_questions:
        print(question, score)
        questions.append(question)
    
    return questions

## RRF algorithm

In [ ]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            # k is a constant smoothing factor that prevents documents from being overly penalized for being far down the list
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

## Generation

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template_RAG_generation = """   
Comportati come un assistente che risponde alle domande del cliente.   
Rispondi alla domanda basandoti solo sui seguenti documenti: {context}.

Rispondi in modo conciso e chiaro, spiegando passo passo al cliente le azioni necessarie da effettuare.   
Se possibile, dai indicazioni dettagliate al cliente, su come risolvere il problema o effettuare l'azione desiderata. 
Evita troppe ripetizioni nella risposta fornita.
Quando spieghi che cosa è o cosa significa un certo elemento richiesto, non parlarne come se fosse un problema.

In caso di più domande rispondi solo a quelle inerenti alla documentazione e rimani a disposizione per altre domande sull'argomento,
specificando, invece, che le altre domande non sono state trovate pertinenti in questo contesto.

Domanda relativa al software Panthera: {question} 
"""

generation_prompt = ChatPromptTemplate.from_template(template_RAG_generation)

## RAG pipeline

In [ ]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
basic_rag_chain = (
    {"context": rerank_questions | retriever.map() | reciprocal_rank_fusion | format_docs, "question": RunnablePassthrough()}
    | generation_prompt
    | model_gemini
    | StrOutputParser()
)

# Evaluate the Advanced RAG pipeline on a small testset

In [ ]:
import pandas as pd

# Load the saved CSV file
eval_df = pd.read_csv('filtered_matching_questions.csv')

# Display the first few rows of the loaded DataFrame
display(eval_df)

In [ ]:
import importlib
import E2E_Evaluation_metrics
importlib.reload(E2E_Evaluation_metrics)
from E2E_Evaluation_metrics import RAGEvaluator
from E2E_Evaluation_metrics import SemScoreQueryRewriting

evaluator = RAGEvaluator()
semscore = SemScoreQueryRewriting()

In [ ]:
import pandas as pd

def generate_answers(generation_chain, df, model_name, chunking_type, preprocessing, retriever, techniques):
    # Create a copy of the original dataframe to avoid modifying it
    new_df = df.copy()
    new_df['generated_answer'] = None
    new_df['model'] = None
    new_df['chunking'] = None
    new_df['preprocessing'] = None
    new_df['retriever'] = None
    new_df['advanced_techniques'] = None

    # Iterate through the dataframe and generate answers
    for idx, elem in new_df.iterrows():
        question = elem["question"]
        new_df.at[idx, 'generated_answer'] = generation_chain.invoke(question) 
        new_df.at[idx, 'model'] = model_name
        new_df.at[idx, 'chunking'] = chunking_type
        new_df.at[idx, 'preprocessing'] = preprocessing
        new_df.at[idx, 'retriever'] = retriever
        new_df.at[idx, 'advanced_techniques'] = techniques

    return new_df

In [ ]:
df = generate_answers(basic_rag_chain, eval_df, 'Gemini', 'Recursive', 'Simple Preprocessing', 'Dense-6', 'No advanced techniques')

In [ ]:
columns_to_drop = ['BLEU', 'ROUGE-2', 'ROUGE-L', 'BERT P', 'BERT R', 'Perplexity', 'Diversity']

def evaluate_responses(eval_df, evaluator):
    results = []
    for _, row in eval_df.iterrows():
        response = row['generated_answer']
        reference = row['answer']
        
        # Check if either response or reference is empty, and skip this row
        if not response or not reference:
            continue
        
        # Evaluate and store the results
        evaluation = evaluator.evaluate_all(response, reference)
        results.append(evaluation)
    
    # Convert results to a DataFrame
    eval_df = pd.DataFrame(results)
    return eval_df


def process_evaluation_and_metrics(data_frame, model_name, evaluator = evaluator, semscore = semscore, columns_to_drop = columns_to_drop):
    """
    Evaluate responses, compute semantic scores, and merge results into a DataFrame.

    Parameters:
    - data_frame (pd.DataFrame): The input DataFrame with original and rewritten questions.
    - evaluator (object): The evaluation object to compute BLEU, ROUGE, etc.
    - semscore (object): The semantic score computation object.
    - model_name (str): Name of the model for semantic similarity scoring.
    - columns_to_drop (list): List of columns to drop from the evaluated DataFrame.

    Returns:
    - pd.DataFrame: Updated DataFrame with evaluation metrics and semantic scores.
    """
    # Step 1: Evaluate responses
    eval_df = evaluate_responses(data_frame, evaluator)
    
    # Step 2: Drop unnecessary columns
    eval_df = eval_df.drop(columns=columns_to_drop, errors="ignore")

    # Step 3: Compute semantic scores
    reference = "answer"
    response = "generated_answer"
    cosine_similarities_bge, _ = semscore.compute_sem_score(data_frame, model_name=model_name, reference=reference, response=response)
    eval_df["SemScore"] = cosine_similarities_bge["Cosine_Similarity"]

    # Step 4: Merge original DataFrame with evaluation metrics
    merged_df = pd.concat([data_frame, eval_df], axis=1)

    return merged_df

In [ ]:
eval_df = process_evaluation_and_metrics(
    data_frame=df, 
    model_name='BAAI/bge-m3'
)

display(eval_df)

In [ ]:
# Optionally, save the filtered dataframe to a CSV file
eval_df.to_csv('ResultsOnTestset/GeminiBaselineRecursiveChunkingSimplePreprocessing.csv', index=False)

In [ ]:
import pandas as pd

def compute_average_value(df, output_file):
    # Compute the averages
    mean_rouge = df['ROUGE-1'].mean()
    mean_bert = df['BERT F1'].mean()
    mean_sem = df['SemScore'].mean()

    # Get model and other details
    model = df['model'].unique()
    chunking = df["chunking"].unique()
    preprocessing = df['preprocessing'].unique()
    retriever = df['retriever'].unique()
    advanced_techniques = df["advanced_techniques"].unique()

    # Create a dictionary of the results
    results = {
        'Model': model,
        'Chunking': chunking,
        'Preprocessing': preprocessing,
        'Retriever': retriever,
        'Advanced Techniques': advanced_techniques,
        'Mean ROUGE-1': mean_rouge,
        'Mean BERT F1': mean_bert,
        'Mean SemScore': mean_sem
    }

    # Convert the dictionary to a DataFrame
    results_df = pd.DataFrame([results])

    # Append the results to the CSV file (if it exists, otherwise create a new one)
    results_df.to_csv(output_file, mode='a', header=not pd.io.common.file_exists(output_file), index=False)

    # Print the results (optional)
    print("Model:", model, "with chunking of type:", chunking, "that uses", 
          preprocessing, retriever, advanced_techniques)
    print(f"Mean ROUGE-1: {mean_rouge}")
    print(f"Mean BERT F1: {mean_bert}")s
    print(f"Mean SemScore: {mean_sem}")

In [ ]:
compute_average_value(eval_df, "ResultsMeanScore/Improved.csv")